In [1]:
import geopandas as gpd
import pandas as pd

In [54]:
# Join Station Characteristics
gdf = gpd.read_file('../RawData/Seoul_subway.shp', encoding = 'utf8')
df = pd.read_csv('../RawData/전체_도시철도역사정보_20230915.csv', encoding = 'cp949')

# Clean gdf and create new columns
gdf = gdf.drop(columns = ['UFID', 'BJCD', 'SCLS', 'FMTA'])
gdf.reset_index(inplace = True)

# Clean df
df = df.drop(columns = ['노선번호', '영문역사명', '한자역사명', '환승노선번호',
       '환승노선명', '역위도', '역경도', '운영기관명', '역사도로명주소', '역사전화번호', '데이터기준일자'])
df.rename(columns = {'역사명': 'NAME'},  inplace = True)
df = df[~df['노선명'].isin(['부산 도시철도 3호선', '부산 도시철도 1호선', '동해선','서해선', '부산 도시철도 2호선', '대전 도시철도 1호선'])]

# Join line info and transfer
def station_name_cut(x):
    x = x.split('(')[0]
    if x[-1] == '역':
        return x[:-1]
    else: return x
gdf['NAME'] = gdf['NAME'].apply(station_name_cut)
df['NAME'] = df['NAME'].apply(station_name_cut)
merged_gdf = gdf.merge(df, on = 'NAME', how = 'left')

merged_gdf.at[175, '역번호'] = '0215'
merged_gdf.at[175, '노선명'] = '2호선'
merged_gdf.at[175, '환승역구분'] = '도시철도 일반역'

merged_gdf.at[328, '역번호'] = 'I1104'
merged_gdf.at[328, '노선명'] = '4호선'
merged_gdf.at[328, '환승역구분'] = '도시철도 환승역'

# Transfer
def transfer(x):
    if x.split(' ')[-1] == '환승역': return 1
    else: return 0
merged_gdf['transfer'] = merged_gdf['환승역구분'].apply(transfer)
merged_gdf.drop_duplicates(subset = 'index', inplace = True)
merged_gdf.drop(columns = ['역번호', '환승역구분'], inplace = True)

# In seoul
def inseoul_col(x, inseoul_list):
    if x in inseoul_list:
        return 0
    else: return 1
inseoul = ['2호선', '수도권  도시철도 9호선', '5호선', '6호선', '7호선', '수도권 경량도시철도 신림선', '우이신설선']
merged_gdf['outer'] = merged_gdf['노선명'].apply(lambda x: inseoul_col(x, inseoul))

# Terminal
def terminal_col(x, terminal_list):
    if x in terminal_list:
        return 1
    else: return 0
terminal = ['까치산', '신설동', '오금', '방화', '마천', '신내', '암사', '개화', '신사', '북한산우이', '서울역', '샛강',  '관악산']
merged_gdf['terminal'] = merged_gdf['NAME'].apply(lambda x: terminal_col(x, terminal))

# CBD
def CBD_col(x, cbd_list):
    if x in cbd_list:
        return 1
    else: return 0
cbd = ['1호선', '2호선', '5호선', '수도권  도시철도 9호선', '2호선', '신분당선']
merged_gdf['cbd'] = merged_gdf['노선명'].apply(lambda x: CBD_col(x, cbd))


In [55]:
merged_gdf

,index,NAME,geometry,노선명,transfer,outer,terminal,cbd
0,0,복정,POINT (966990.665 1941356.140),8호선,1,1,0,0
2,1,마천,POINT (969361.329 1944074.146),5호선,0,0,1,1
3,2,거여,POINT (968560.267 1943818.933),5호선,0,0,0,1
4,3,개롱,POINT (967671.846 1944410.263),5호선,0,0,0,1
5,4,장지,POINT (966920.018 1942299.169),8호선,0,1,0,0
...,...,...,...,...,...,...,...,...
395,297,서원,POINT (949867.555 1942259.556),수도권 경량도시철도 신림선,0,0,0,0
396,298,서울대벤처타운,POINT (949927.215 1941571.122),수도권 경량도시철도 신림선,0,0,0,0
397,299,관악산,POINT (950950.800 1941203.801),수도권 경량도시철도 신림선,0,0,1,0
398,300,봉천,POINT (950660.687 1942716.953),2호선,0,0,0,1


In [ ]:
['1호선', '2호선', '5호선', '수도권  도시철도 9호선', '2호선', '신분당선']

In [56]:
save_gdf = merged_gdf.drop(columns = ['노선명'])
save_gdf.to_file('../Data/SubwayInfo.shp', encoding = 'utf8')

In [7]:
merged_gdf.at[175, '역번호'] = '0215'
merged_gdf.at[175, '노선명'] = '2호선'
merged_gdf.at[175, '환승역구분'] = '도시철도 일반역'

merged_gdf.at[245, '역번호'] = 'I1104'
merged_gdf.at[175, '노선명'] = '4호선'
merged_gdf.at[175, '환승역구분'] = '도시철도 환승역'

In [11]:
def station_name_cut(x):
    x = x.split('(')[0]
    if x[-1] == '역':
        return x[:-1]
    else: return x
gdf['NAME'] = gdf['NAME'].apply(station_name_cut)
df['NAME'] = df['NAME'].apply(station_name_cut)

merged_gdf = gdf.merge(df, on = 'NAME', how = 'left')
gdf

,index,NAME,geometry
0,0,복정,POINT (966990.665 1941356.140)
1,1,마천,POINT (969361.329 1944074.146)
2,2,거여,POINT (968560.267 1943818.933)
3,3,개롱,POINT (967671.846 1944410.263)
4,4,장지,POINT (966920.018 1942299.169)
...,...,...,...
297,297,서원,POINT (949867.555 1942259.556)
298,298,서울대벤처타운,POINT (949927.215 1941571.122)
299,299,관악산,POINT (950950.800 1941203.801)
300,300,봉천,POINT (950660.687 1942716.953)


In [63]:
df0 = pd.read_csv('../RawData/2022년기준_2020년_산업분류별(10차_대분류)_종사자수.txt', sep = '^', header = None)

In [68]:
df1 = df0.merge(df0.groupby(1).sum().reset_index(), on = 1)

C:\Users\Chulwoong Park\AppData\Local\Temp\ipykernel_23136\1713394365.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df1 = df0.merge(df0.groupby(1).sum().reset_index(), on = 1)


In [70]:
df1.drop_duplicates(1).to_csv('../RawData/employments.csv', encoding = 'utf8')

In [71]:
df1

,0_x,1,2,3_x,0_y,3_y
0,2020,31104530020201,cp_bem_017,13.0,22220,148.0
1,2020,31104530020201,cp_bem_018,12.0,22220,148.0
2,2020,31104530020201,cp_bem_016,58.0,22220,148.0
3,2020,31104530020201,cp_bem_014,NaN,22220,148.0
4,2020,31104530020201,cp_bem_007,27.0,22220,148.0
...,...,...,...,...,...,...
806535,2020,37100110020202,cp_bem_999,0.0,2020,0.0
806536,2020,38080540010303,cp_bem_999,0.0,2020,0.0
806537,2020,38080540010702,cp_bem_999,0.0,2020,0.0
806538,2020,38112520010105,cp_bem_999,0.0,2020,0.0
